In [ ]:
!pip install datasets scikit-learn

In [ ]:
!pip3 install spacy
!python3 -m spacy download ru_core_news_sm

In [45]:
from typing import List, Tuple
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import spacy
from datasets import load_dataset

In [46]:
def load_sub200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [47]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [48]:
train_data, val_data, test_data, classes_list = load_sub200_ru()

In [49]:
print(f'Categories: {classes_list}')

Categories: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [50]:
print(len(train_data[0]))
print(len(train_data[1]))

701
701


In [51]:
print(len(val_data[0]))
print(len(val_data[1]))

99
99


In [52]:
print(len(test_data[0]))
print(len(test_data[1]))

204
204


In [53]:
nlp = spacy.load("ru_core_news_sm")

In [54]:
print(train_data[0][0])
print(normalize_text(train_data[0][0], nlp))

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.
турция с <NUM> сторона окружить море на запад эгейский на север чёрный и на юг средиземный


In [55]:
print(val_data[0][0])
print(normalize_text(val_data[0][0], nlp))

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.
если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость


In [56]:
print(test_data[0][0])
print(normalize_text(test_data[0][0], nlp))

Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.
мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [57]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability
print(f"Maximal document frequency of term is {max_df}.")

Maximal document frequency of term is 0.9714285714285714.


In [67]:
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1)),
    ('classifier', LogisticRegression(solver='saga', max_iter=100, random_state=42))
])

In [91]:
cv = GridSearchCV(
    estimator=classifier,
    # ОРИГИНАЛЬНЫЙ КОД (ЧАСТИЧНО УТЕРЯН, НЕ РАБОТАЕТ):
    # param_grid={
    #     'cls__C': np.logspace(-4, 4, 9),
    #     'cls__penalty': ['l1', 'l2'],
    #     'vectorizer__ngram_range': [(1, 1), (1, 2)],
    # },

    # КОД, ПРЕДЛОЖЕННЫЙ НЕЙРОНКОЙ (РАБОТАЕТ, НО В ОРИГИНАЛЕ БЫЛ ДРУГОЙ):
    param_grid={
        'vectorizer__max_df': np.arange(0.80, 1.00, 0.01),
        'vectorizer__min_df': np.arange(0.001, 0.01, 0.001),
    },
    scoring='f1_macro',
    n_jobs=-1,
    verbose=True
)

In [ ]:
cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

In [93]:
print('Best parameters:')
print(cv.best_params_)

Best parameters:
{'vectorizer__max_df': 0.8, 'vectorizer__min_df': 0.006}


In [94]:
print('Sest Fl-macro:')
print(cv.best_score_)

Sest Fl-macro:
0.4577708076667141


In [95]:
print('Vocabulary size is:')
print(len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_))

Vocabulary size is:
484


In [96]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       1.00      0.22      0.36         9
         geography       0.80      0.50      0.62         8
            health       1.00      0.27      0.43        11
          politics       0.58      0.50      0.54        14
science/technology       0.48      0.84      0.61        25
            sports       0.67      0.50      0.57        12
            travel       0.42      0.50      0.45        20

          accuracy                           0.54        99
         macro avg       0.71      0.48      0.51        99
      weighted avg       0.63      0.54      0.52        99

